In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.collections import LineCollection

from utils import DatabaseConnection

In [ ]:
from shapely.geometry import Point
from shapely.geometry import box

start_rect = box(13.416354, 52.521752, 13.416705, 52.521961)
end_rect = box(13.416912, 52.522261, 13.417263, 52.52247)

def crop_intersection_SimRa(x):
    mask_first = x.coords.apply(lambda x: start_rect.contains(Point(x)))
    mask_end = x.coords.apply(lambda x: end_rect.contains(Point(x)))
    try:
        first = mask_first[mask_first==True].index[0]
        last = mask_end[mask_end==True].index[-1]
        return x.loc[first:last]
    except:
        print('check')
        
def to_float_arr(x):
    res = []
    for c in x:
        res.append(float(c))
    return res


In [ ]:
ped_files = ['VM2_-2112701535', 'VM2_-217686115','VM2_-1247665811','VM2_-104300786', 'VM2_330973206', 'VM2_1100569031', 'VM2_421371629', 'VM2_421371629', 'VM2_1476499235', 'VM2_-1451152685', 'VM2_-1523872256']


with DatabaseConnection() as cur:
    cur.execute("""
SELECT filename,
       json_array_elements_text(st_asgeojson(geom_raw) :: json -> 'coordinates') AS coordinates,
       unnest(timestamps)                             timestamps,
       unnest(velos)                                  velos
FROM ride
where st_intersects(geom, st_setsrid(
        st_makebox2d(st_makepoint(13.416354, 52.521752), st_makepoint(13.416705, 52.521961)), 4326))
  AND st_intersects(geom, st_setsrid(
        st_makebox2d(st_makepoint(13.416912, 52.522261), st_makepoint(13.417263, 52.52247)), 4326))
  AND timestamps[1] < '2021-01-01 00:00:00'""")
    res = cur.fetchall()
    df = pd.DataFrame(res, columns=['filename', 'coords', 'timestamps', 'velos'])
    df = df[~df.coords.isnull()]
    df = df[~df['filename'].isin(ped_files)]
    df.coords = df.coords.apply(lambda x: x.replace('[', '').replace(']', '').split(','))
    df.coords = df.coords.apply(lambda x: to_float_arr(x))
    
    grouped = df.groupby('filename', as_index=False)
    df_simra = grouped.apply(lambda x: crop_intersection_SimRa(x))


In [ ]:
# from matplotlib import rc

# rc('font', **{'family': 'serif', 'serif': ['Computer Modern'], 'size': 16})
# rc('text', usetex=True)

In [ ]:
df_simra

In [ ]:
df_simra['lon'], df_simra['lat'] = zip(*df_simra.coords.values)

fig, ax = plt.subplots(figsize=(12, 12))
ax.set_aspect('equal')
# Create a continuous norm to map from data points to colors
norm = plt.Normalize(df_simra.velos.min(), df_simra.velos.max())

df_simra_grouped = df_simra.groupby('filename')
for ride_group_name in df_simra_grouped.groups:
    df_ride_group = df_simra_grouped.get_group(ride_group_name)
    points = np.array([df_ride_group.lon, df_ride_group.lat]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    lc = LineCollection(segments, cmap='viridis', norm=norm)
    # Set the values used for colormapping
    lc.set_array(df_ride_group.velos.values)
    lc.set_linewidth(2)
    line = ax.add_collection(lc)

ax.set_xlim(min(df_simra.lon), max(df_simra.lon))
ax.set_ylim(min(df_simra.lat), max(df_simra.lat))

axcb = fig.colorbar(lc, ax=ax, shrink=0.8, pad=0.1)
axcb.set_label('Velocity in m/s')

ax.xaxis.set_major_locator(ticker.LinearLocator(4))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
ax.yaxis.set_major_locator(ticker.LinearLocator(4))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
ax.set_xlabel('Longitude in decimal degrees')
ax.set_ylabel('Latitude in decimal degrees')


plt.show()
